# Tables

This notebook creates all tables included in the publication. Also creates the csv files in the data repo.

In [1]:
import pickle
import sys
import matplotlib.pyplot as plt
import numpy as np
from tqdm.auto import tqdm
import os
import pandas as pd
import datetime
import matplotlib.ticker as ticker
sys.path.append("../../")
sys.path.append("../../covid19_inference")
sys.path.append("../")

import covid19_soccer
from covid19_soccer.plot.utils import get_from_trace
import covid19_inference as cov19
from header_plotting import *

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
def load(fstr):
    with open(fstr, "rb") as f:
         return pickle.load(f)

countries = ['England', 'Czechia', 'Scotland', 'Spain', 'Italy', 'Slovakia',
       'Germany', 'Austria', 'Belgium', 'France', 'Portugal',
       'Netherlands']

traces,traces_without_soccer,traces_primary_soccer, models, dls = [], [], [], [],[]

for country in tqdm(countries):
    #'UEFA-beta=False-country=England-offset_games=0-draw_delay=True-weighted_alpha_prior=0-prior_delay=-1-width_delay_prior=0.1-sigma_incubation=-1.0-median_width_delay=1.0-tune=200-draws=300-max_treedepth=10.pickled'
    model = None
    fstr=lambda tune, draws, max_treedepth: (f"/data.nst/smohr/covid19_soccer_data/main_traces/run"+
        f"-beta=False"+
        f"-country={country}"+
        f"-offset_data=0"+
        f"-prior_delay=-1"+
        f"-median_width_delay=1.0"+
        f"-interval_cps=10.0"+
        f"-f_fem=0.33"+
        f"-len=normal"+
        f"-abs_sine=False"+
        f"-t={tune}"+
        f"-d={draws}"+
        f"-max_treedepth={max_treedepth}.pkl")
    
    #print(fstr(4000, 8000, 12))
    if os.path.exists(fstr(4000, 8000, 12)):
        try:
            model, trace = load(fstr(4000, 8000, 12))
            print(f"Use 8000 sample runs for {country}")
        except:
            pass
    if model is None and os.path.exists(fstr(2000, 4000, 12)):
        try:
            model, trace = load(fstr(2000, 4000, 12))
            print(f"Use 4000 sample runs for {country}")
        except:
            pass
    if model is None and os.path.exists(fstr(1000, 1500, 12)):
        try: 
            model, trace = load(fstr(1000, 1500, 12))
            print(f"Use 1500 sample runs for {country}")
        except:
            pass
    if model is None:
        print(" not found")
        continue
    
    # Remove chains with likelihood larger than -200, should only be the case for 2 chains in France
    mask = (np.mean(trace.sample_stats.lp, axis=1)>-200)
    trace.posterior = trace.posterior.sel(chain=~mask)
    dl = covid19_soccer.dataloader.Dataloader_gender(countries=[country])
    models.append(model)
    traces.append(trace)
    dls.append(dl)
    
    trace_without_soccer, trace_primary_soccer = load(f"/data.nst/smohr/covid19_soccer_data/primary_and_subsequent/{country}.pkl")
    traces_without_soccer.append(trace_without_soccer)
    traces_primary_soccer.append(trace_primary_soccer)
    
# Load overall data
trace_overall = load("/data.nst/smohr/covid19_soccer_data/trace_overall_effekt.pkl")
alpha_overall = load("/data.nst/smohr/covid19_soccer_data/trace_overall_alpha.pkl")


  0%|          | 0/12 [00:00<?, ?it/s]

Use 4000 sample runs for England
Use 4000 sample runs for Czechia
Use 4000 sample runs for Scotland
Use 4000 sample runs for Spain
Use 4000 sample runs for Italy
Use 4000 sample runs for Slovakia
Use 4000 sample runs for Germany
Use 4000 sample runs for Austria
Use 4000 sample runs for Belgium
Use 4000 sample runs for France
Use 4000 sample runs for Portugal
Use 4000 sample runs for Netherlands


In [4]:
infections_no = []
infections_primary = []
infections_secondary = []
for i, country in enumerate(countries):
    shape = trace_without_soccer.predictions["new_cases"].to_numpy().shape
    x = pd.date_range(models[i].sim_begin, models[i].sim_end)
    y0 = get_from_trace("new_cases",traces_without_soccer[i],from_type="predictions")
    y1 = get_from_trace("new_cases",traces_primary_soccer[i],from_type="predictions") - y0
    y2 = get_from_trace("new_cases",traces[i]) - y1 - y0
    
    begin = datetime.datetime(2021, 6, 11)
    end = datetime.datetime(2021, 7, 31)
    i_begin = (begin - model.sim_begin).days
    i_end = (end - model.sim_begin).days + 1  # inclusiv last day

    y0 = np.sum(y0[..., i_begin:i_end, :], axis=-2)
    y1 = np.sum(y1[..., i_begin:i_end, :], axis=-2)
    y2 = np.sum(y2[..., i_begin:i_end, :], axis=-2)

    infections_no.append(y0)
    infections_primary.append(y1)
    infections_secondary.append(y2)

In [5]:
data_primary  = pd.DataFrame()
means = []
for i,country in enumerate(countries):
    # compute fraction of infected people
    t_numpy = infections_primary[i] / (infections_no[i] + infections_primary[i] + infections_secondary[i]) * 100

    male = np.stack(
        (t_numpy[:, 0], np.zeros(t_numpy[:, 0].shape)), axis=1
    )
    female = np.stack(
        (t_numpy[:, 1], np.ones(t_numpy[:, 1].shape)), axis=1
    )
    # Create dataframe for plotting
    temp = pd.DataFrame(np.concatenate((male, female)), columns=["percentage_primary", "gender"])
    temp["gender"] = pd.cut(
        temp["gender"], bins=[-1, 0.5, 1], labels=["male", "female"]
    )
    temp["country"] = country
    data_primary = pd.concat([data_primary, temp])
    means.append(np.mean(temp["percentage_primary"]))
country_order_primar = np.argsort(means)[::-1]


data_subsequent  = pd.DataFrame()
means = []
for i,country in enumerate(countries):
    # compute fraction of infected people
    t_numpy = infections_secondary[i] / (infections_no[i] + infections_primary[i] + infections_secondary[i]) * 100

    male = np.stack(
        (t_numpy[:, 0], np.zeros(t_numpy[:, 0].shape)), axis=1
    )
    female = np.stack(
        (t_numpy[:, 1], np.ones(t_numpy[:, 1].shape)), axis=1
    )
    # Create dataframe for plotting
    temp = pd.DataFrame(np.concatenate((male, female)), columns=["percentage_subsequent", "gender"])
    temp["gender"] = pd.cut(
        temp["gender"], bins=[-1, 0.5, 1], labels=["male", "female"]
    )
    temp["country"] = country
    data_subsequent = pd.concat([data_subsequent, temp])
    means.append(np.mean(temp["percentage_subsequent"]))
country_order_subs = np.argsort(means)[::-1]


data_primary_and_subsequent = pd.DataFrame()
means = []
for i, country in enumerate(countries):

    # compute fraction
    t_numpy = (infections_primary[i]+infections_secondary[i])/(infections_no[i] + infections_primary[i] + infections_secondary[i]) * 100

    male = np.stack(
        (t_numpy[:, 0], np.zeros(t_numpy[:, 0].shape)), axis=1
    )
    female = np.stack(
        (t_numpy[:, 1], np.ones(t_numpy[:, 1].shape)), axis=1
    )
    # Create dataframe for plotting
    temp = pd.DataFrame(np.concatenate((male, female)), columns=["percentage_primary_and_subsequent", "gender"])
    temp["gender"] = pd.cut(
        temp["gender"], bins=[-1, 0.5, 1], labels=["male", "female"]
    )
    temp["country"] = country
    data_primary_and_subsequent = pd.concat([data_primary_and_subsequent, temp])
    means.append(np.mean(temp["percentage_primary_and_subsequent"]))
country_order_primary_and_subsequent = np.argsort(means)[::-1]

In [6]:
# Compute interesting values as fraction and as incidence
primary_frac = []
for i, country in enumerate(countries):
    temp = data_primary[data_primary["country"] == country].drop(columns=["country","gender"])
    primary_frac.append(temp.to_numpy()[:,0])

subsequent_frac = []
for i, country in enumerate(countries):
    temp = data_subsequent[data_subsequent["country"] == country].drop(columns=["country","gender"])
    primary_frac.append(temp.to_numpy()[:,0])

primary_and_subsequent_frac = []
for i, country in enumerate(countries):
    temp = data_primary_and_subsequent[data_primary_and_subsequent["country"] == country].drop(columns=["country","gender"])
    primary_and_subsequent_frac.append(temp.to_numpy()[:,0])


primary_inci = []
for i, country in enumerate(countries):
    temp = infections_primary[i]
    primary_inci.append(temp / dls[i].population[0]*1e6)

primary_and_subsequent_inci = []
for i, country in enumerate(countries):
    temp = infections_primary[i] + infections_secondary[i]
    primary_and_subsequent_inci.append(temp/ dls[i].population[0]*1e6)

primary_cases = []
for i, country in enumerate(countries):
    temp = infections_primary[i]
    primary_cases.append(temp)

primary_and_subsequent_cases = []
for i, country in enumerate(countries):
    temp = infections_primary[i] + infections_secondary[i]
    primary_and_subsequent_cases.append(temp)
    

## SI Table 1: Fractions

In [7]:
table = []
for i, country in enumerate(countries):
    # Primary infections
    data  = data_primary[data_primary["country"] == country]

    CI = np.percentile(data["percentage_primary"], q = (2.5,50,97.5))
    prob_positive = np.sum(data["percentage_primary"] > 0) / data["percentage_primary"].shape[0] * 100

    # Secondary infections and primary
    data  = data_primary_and_subsequent[data_primary_and_subsequent["country"] == country]
    CI_p_sub = np.percentile(data["percentage_primary_and_subsequent"], q = (2.5,50,97.5))
    
    data  = data_subsequent[data_subsequent["country"] == country]
    CI_sub = np.percentile(data["percentage_subsequent"], q = (2.5,50,97.5))
    
    
    
    # Create table entries
    if prob_positive > 99.9:
        text_prob_pos = "$> 99.9$\%"
    else:
        text_prob_pos = f"{prob_positive:.1f}\%"

    table.append([
        country,
        f"{CI[1]:.1f}\% [{CI[0]:.1f}\%, {CI[2]:.1f}\%]",
        f"{CI_sub[1]:.1f}\% [{CI_sub[0]:.1f}\%, {CI_sub[2]:.1f}\%]",
        f"{CI_p_sub[1]:.1f}\% [{CI_p_sub[0]:.1f}\%, {CI_p_sub[2]:.1f}\%]",
        text_prob_pos,]
    )

#1.3378240349811912	3.239547956846547	5.169654234351223	0.998875
table.append([
    "Avg.",
    f"{3.2}\% [{1.3}\%, {5.2}\%]",
    f"-",
    f"-",
    r"$> 99.9$\%",]
)


In [8]:
from tabulate import tabulate
headers = ['Country', 'Median percentage of primary cases', 'Median percentage of subsequent cases', "Median percentage of primary and subsequent cases",  'Probability that soccer increased cases']
print(tabulate(table, headers, tablefmt="latex_raw"))

csv = pd.DataFrame(columns=headers,data=table)

# Add male and female
csv = csv.set_index("Country")

\begin{tabular}{lllll}
\hline
 Country     & Median percentage of primary cases   & Median percentage of subsequent cases   & Median percentage of primary and subsequent cases   & Probability that soccer increased cases   \\
\hline
 England     & 12.4\% [5.6\%, 22.5\%]               & 36.0\% [27.9\%, 44.7\%]                 & 47.8\% [36.0\%, 62.9\%]                             & $> 99.9$\%                                \\
 Czechia     & 9.7\% [3.3\%, 16.2\%]                & 47.8\% [24.2\%, 58.7\%]                 & 57.7\% [28.7\%, 72.6\%]                             & $> 99.9$\%                                \\
 Scotland    & 3.3\% [1.3\%, 8.1\%]                 & 36.6\% [28.6\%, 43.9\%]                 & 40.8\% [30.9\%, 50.3\%]                             & $> 99.9$\%                                \\
 Spain       & 2.8\% [-1.1\%, 9.2\%]                & 24.1\% [-16.3\%, 60.6\%]                & 26.9\% [-16.9\%, 69.2\%]                            & 91.8\%                           

## SI Table 2: Totals

In [9]:
table_2 = []
for i, country in enumerate(countries):
    
    c = country_order_primar[i]

    row = []
    
    primary_mean, primary_lower, primary_upper = np.percentile(primary_inci[c], q = (50,2.5,97.5), axis=0)
    
    row.append(country)
    for m,l,u in zip(primary_mean, primary_lower, primary_upper):
        row.append(f"{m:.0f} [{l:.0f}, {u:.0f}]")
    
    # Total without gender
    m,l,u  = np.percentile(primary_and_subsequent_cases[i].sum(axis=-1)*1e6/dls[i].population.sum(),q = (50,2.5,97.5), axis=0)
    row.append(f"{m:.0f} [{l:.0f}, {u:.0f}]")
    
    table_2.append(row)

# Add avgs
row = []
row.append("Avg.")


row.append(f"-")
row.append(f"-")
avg_pr_and_su = [2228.29262671, 985.87627683, 3308.00843463]
row.append(f"{avg_pr_and_su[0]:.0f} [{avg_pr_and_su[1]:.0f}, {avg_pr_and_su[2]:.0f}]")
table_2.append(row)

In [10]:
from tabulate import tabulate
headers = ['Country', 'Primary cases per $10^6$ inhibitants (male)', 'Primary cases per $10^6$ inhibitants (female)', "Primary and subsequent cases per $10^6$ inhibitants"]
print(tabulate(table_2, headers, tablefmt="latex_raw"))

\begin{tabular}{llll}
\hline
 Country     & Primary cases per $10^6$ inhibitants (male)   & Primary cases per $10^6$ inhibitants (female)   & Primary and subsequent cases per $10^6$ inhibitants   \\
\hline
 England     & 3595 [2661, 5729]                             & 1686 [1143, 3453]                               & 10600 [8185, 13875]                                   \\
 Czechia     & 94 [40, 142]                                  & 65 [22, 108]                                    & 459 [229, 577]                                        \\
 Scotland    & 1352 [940, 1758]                              & 351 [222, 517]                                  & 7897 [6136, 9529]                                     \\
 Spain       & 594 [-217, 1722]                              & 387 [-160, 1346]                                & 4518 [-2840, 11595]                                   \\
 Italy       & 55 [-121, 227]                                & 27 [-77, 131]                                   & 3

In [16]:
table_2_5 = []
owd = cov19.data_retrieval.OWD(True)

for i, country in enumerate(countries):
    row = []
    c = country_order_primar[i]
    
    if country in ["Scotland","England"]:
        df = owd.data[owd.data["country"] == "United Kingdom"]
    else:
        df = owd.data[owd.data["country"] == country]
    
    cfr1 = df["new_deaths"][begin+datetime.timedelta(days=14):end+datetime.timedelta(days=14)].mean()/df["new_cases"][begin:end].mean()
    cfr2 = df["new_deaths"][begin+datetime.timedelta(days=14):end+datetime.timedelta(days=14)].mean()/df["new_cases"][begin:end].mean()
    print(country)
    print(cfr1)

    primary_mean, primary_lower, primary_upper = np.percentile(primary_cases[c], q = (50,2.5,97.5), axis=0)
    subs_mean, subs_lower, subs_upper = np.percentile(np.sum(primary_and_subsequent_cases[c],axis=-1), q = (50,2.5,97.5), axis=0)
    fat_mean, fat_lower, fat_upper = np.percentile(np.sum(primary_and_subsequent_cases[c],axis=-1)*cfr1, q = (50,2.5,97.5), axis=0)
    
    row.append(country)
    for m,l,u in zip(primary_mean, primary_lower, primary_upper):
        row.append(f"{m:.0f} [{l:.0f}, {u:.0f}]")

    row.append(f"{subs_mean:.0f} [{subs_lower:.0f}, {subs_upper:.0f}]")    
    
    # Fatility
    row.append(f"{fat_mean:.0f} [{fat_lower:.0f}, {fat_upper:.0f}]")
    
    table_2_5.append(row)
    
def reduce_samples(curr_list):
    number_of_samples = 2000
    for i, array in enumerate(curr_list):
        array = array[np.random.choice(array.shape[0], number_of_samples, replace = False),...]
        curr_list[i] = array
    return np.array(curr_list)

row= []
pc = reduce_samples(primary_cases)
psc = reduce_samples(primary_and_subsequent_cases)

primary_mean, primary_lower, primary_upper = np.percentile(np.sum(pc,axis=0), q = (50,2.5,97.5), axis=0)
subs_mean, subs_lower, subs_upper = np.percentile(np.sum(psc,axis=(0,-1)), q = (50,2.5,97.5), axis=0)
fat_mean, fat_lower, fat_upper = np.percentile(np.sum(psc,axis=(0,-1))*0.002, q = (50,2.5,97.5), axis=0)

row.append("Total")
for m,l,u in zip(primary_mean, primary_lower, primary_upper):
    row.append(f"{m:.0f} [{l:.0f}, {u:.0f}]")

row.append(f"{subs_mean:.0f} [{subs_lower:.0f}, {subs_upper:.0f}]")    

# Fatility
row.append(f"{fat_mean:.0f} [{fat_lower:.0f}, {fat_upper:.0f}]")

table_2_5.append(row)

INFO     [covid19_inference.data_retrieval.retrieval] Successfully loaded OurWorldinData.csv.gz from /tmp/covid19_data/, skipping download.


England
0.0021627950713752706
Czechia
0.01217118560692769
Scotland
0.0021627950713752706
Spain
0.0023746282675526
Italy
0.009540668119099493
Slovakia
0.01327433628318584
Germany
0.020787527483509892
Austria
0.005539753113694734
Belgium
0.002672685157688424
France
0.004407047253200326
Portugal
0.004151980141167325
Netherlands
0.0009009191399765154


In [17]:
from tabulate import tabulate
headers = ['Country', 'Primary cases (male)', 'Primary cases (female)', "Primary and subsequent cases", "Number of associated deaths"]
print(tabulate(table_2_5, headers, tablefmt="latex_raw"))

\begin{tabular}{lllll}
\hline
 Country     & Primary cases (male)   & Primary cases (female)   & Primary and subsequent cases   & Number of associated deaths   \\
\hline
 England     & 93619 [69591, 145127]  & 43872 [29946, 87030]     & 567280 [436870, 747399]        & 1227 [945, 1616]              \\
 Czechia     & 494 [215, 753]         & 346 [116, 558]           & 4920 [2455, 6182]              & 60 [30, 75]                   \\
 Scotland    & 3478 [2444, 4481]      & 908 [574, 1320]          & 41720 [31766, 50146]           & 90 [69, 108]                  \\
 Spain       & 13570 [-4463, 40212]   & 8870 [-3339, 31389]      & 211952 [-122694, 546650]       & 503 [-291, 1298]              \\
 Italy       & 1535 [-3399, 6718]     & 750 [-2219, 3824]        & 17810 [-243916, 79338]         & 170 [-2327, 757]              \\
 Slovakia    & 21 [-87, 100]          & 11 [-47, 67]             & 320 [-1809, 1087]              & 4 [-24, 14]                   \\
 Germany     & 618 [-629, 1460] 

## SI Table 3: Number of games hosted and played

In [88]:
table_3 = []
df = pd.read_csv("../../data/em_game_data.csv",header=2)
df = df[~df["id"].str.contains("a")] # Filter extra games we added for validation these are suffixed with a
df[" date"]= pd.to_datetime(df[" date"])

country2location = {
    "GB-ENG":"London",
    "IT":"Rome",
    "AZ":"Baku",
    "DE":"Munich",
    "RU":"Saint Petersburg",
    "HU":"Budapest",
    "ES":"Seville",
    "RO":"Bucharest",
    "NL":"Amsterdam",
    "GB-SCT":"Glasgow",
    "DK":"Copenhagen"
}
country_order_primar = list(range(len(countries)))
for i, country in enumerate(np.array(countries)):
    c = country_order_primar[i]
    iso2 = dls[c].countries_iso2[0]
    gamesByTeam = df[df[' team1'].str.contains(iso2) | df[' team2'].str.contains(iso2)] 
    
    
    if iso2 in country2location:
        location = country2location[iso2]
        gamesHosted = df[df[' location'].str.contains(location)]
    else:
        location = "placeholder"
        gamesHosted = np.zeros((0,8))
    
    union = df[df[' team1'].str.contains(iso2) | df[' team2'].str.contains(iso2) |  df[' location'].str.contains(location)]
    row = []
    row.append(country)
    row.append(gamesByTeam.shape[0])
    row.append(gamesHosted.shape[0])
    row.append(union.shape[0])
    row.append((gamesByTeam[" date"].max()-gamesByTeam[" date"].min()).days)
    
    table_3.append(row)

In [89]:
from tabulate import tabulate
headers = ['Country', 'Matches played', 'Matches hosted', "Union", "Days in the tournament"]
print(tabulate(table_3, headers, tablefmt="latex_raw"))

\begin{tabular}{lrrrr}
\hline
 Country     &   Matches played &   Matches hosted &   Union &   Days in the tournament \\
\hline
 England     &                7 &                8 &       9 &                       28 \\
 Czechia     &                5 &                0 &       5 &                       19 \\
 Scotland    &                3 &                4 &       5 &                        8 \\
 Spain       &                6 &                4 &       7 &                       22 \\
 Italy       &                7 &                4 &       8 &                       30 \\
 Slovakia    &                3 &                0 &       3 &                        9 \\
 Germany     &                4 &                4 &       5 &                       14 \\
 Austria     &                4 &                0 &       4 &                       13 \\
 Belgium     &                5 &                0 &       5 &                       20 \\
 France      &                4 &                0 & 

## SI Table 4: Delta R Mean and alpha value for each game and country

In [18]:
table_4 = []
for i,country in enumerate(countries):

    if i == 0:
        alpha_mean = get_from_trace("overall_alpha",alpha_overall)
        
        row = []
        row.append("Avg.")
        mean, lower, upper = np.percentile(alpha_mean,q=[50,2.5,97.5])
        row.append(f"{mean:.2f} [{lower:.2f}, {upper:.2f}]") 

        table_4.append(row) 
    
    alpha_mean = get_from_trace("alpha_mean",traces[i])
    delay = get_from_trace("delay",traces[i])
    
    row = []
    row.append(country)
    mean, lower, upper = np.percentile(alpha_mean,q=[50,2.5,97.5])
    row.append(f"{mean:.2f} [{lower:.2f}, {upper:.2f}]") 
    mean, lower, upper = np.percentile(delay,q=[50,2.5,97.5])
    row.append(f"{mean:.2f} [{lower:.2f}, {upper:.2f}]")   
    table_4.append(row)
    

In [19]:
headers = ['Country', "Delta R match mean", "Delay"]
print(tabulate(table_4, headers, tablefmt="latex_raw"))

\begin{tabular}{lll}
\hline
 Country     & Delta R match mean   & Delay             \\
\hline
 Avg.        & 0.46 [0.18, 0.75]    &                   \\
 England     & 0.75 [0.01, 1.66]    & 4.55 [4.36, 4.94] \\
 Czechia     & 1.26 [-0.50, 3.19]   & 5.53 [4.75, 6.32] \\
 Scotland    & 1.09 [-2.77, 4.69]   & 3.52 [3.35, 3.74] \\
 Spain       & 0.37 [-0.72, 1.83]   & 6.91 [5.43, 7.82] \\
 Italy       & 0.28 [-1.11, 1.79]   & 5.51 [3.96, 7.11] \\
 Slovakia    & 0.32 [-2.27, 2.56]   & 5.00 [3.67, 7.28] \\
 Germany     & 0.33 [-0.62, 1.12]   & 6.82 [5.69, 8.43] \\
 Austria     & 0.28 [-0.90, 1.45]   & 4.58 [3.46, 6.37] \\
 Belgium     & 0.11 [-0.61, 0.92]   & 5.09 [3.71, 6.69] \\
 France      & 0.30 [-0.46, 0.97]   & 3.68 [3.13, 4.46] \\
 Portugal    & -0.02 [-1.33, 1.34]  & 5.49 [4.30, 6.55] \\
 Netherlands & -0.74 [-3.30, 1.36]  & 5.70 [4.28, 6.00] \\
\hline
\end{tabular}
